In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('augmented_train.csv')
df.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
3,fdcd1bd867,From Cockpit Country to St. Ann's Bay,From St. Ann's Bay to Cockpit Country.,en,English,2
4,7cfb3d272c,"Look, it's your skin, but you're going to be i...",The boss will fire you if he sees you slacking...,en,English,1


In [3]:
df.shape

(101367, 6)

In [4]:
df['language'].value_counts()

English       6870
Chinese       6822
Arabic        6802
French        6780
Swahili       6770
Urdu          6760
Vietnamese    6758
Russian       6752
Hindi         6747
Greek         6744
Thai          6742
Spanish       6732
Turkish       6702
German        6702
Bulgarian     6684
Name: language, dtype: int64

In [5]:
df['premise'] = df['premise'].str.replace('.', '',regex=True)
df['premise'] = df['premise'].str.replace('http\S+|www.\S+', '',regex=True)

df['hypothesis'] = df['hypothesis'].str.replace('.', '',regex=True)
df['hypothesis'] = df['hypothesis'].str.replace('http\S+|www.\S+', '',regex=True)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

shuffled_df = shuffle(df)

train, test = train_test_split(
                    shuffled_df, train_size = 0.8, test_size = 0.2, shuffle=True, 
                    stratify = shuffled_df['label'])

In [7]:
del df, shuffled_df

In [8]:
x_train = train.drop(['label', 'id'], axis = 1)
y_train = train['label']

x_test = test.drop(['label', 'id'], axis = 1)
y_test = test['label']

In [9]:
x_train.shape

(81093, 4)

In [10]:
y_train.shape

(81093,)

#### For CLS and word level features

In [47]:
# import torch
# from transformers import XLMRobertaTokenizer, XLMRobertaModel
# tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
# model = XLMRobertaModel.from_pretrained("xlm-roberta-base")
# from tqdm import tqdm
# def extract_xlm_r(list_of_sentences=[],batch_size=16):
#     cls_hidden_states = []
#     train_word_vectors = []
#     if len(list_of_sentences)<batch_size: #go longer ones we should batch
#         tknzed= tokenizer(list_of_sentences, return_tensors="pt",max_length=max_length, padding = "max_length", truncation = True)
#         b=model(**tknzed)
#         cls_hidden_states = torch.squeeze(b.last_hidden_state[:,0,:])
#         train_word_vectors = b.last_hidden_state[:, 1:, :]
#     else:
#         for i in tqdm(range(0,len(list_of_sentences),batch_size)):    
#             tknzed= tokenizer(list_of_sentences[i:min(len(list_of_sentences),i+batch_size)],  max_length = max_length, return_tensors="pt", padding="max_length", truncation = True)
#             b=model(**tknzed)
#             cls_hidden_states.append(torch.squeeze(b.last_hidden_state[:,0,:]))
#             train_word_vectors.append(b.last_hidden_state[:, 1:, :])
# #             print(b.last_hidden_state[:,0,:].shape)
# #             print(b.last_hidden_state[:,1:,:].shape)
#     return cls_hidden_states, train_word_vectors

In [12]:
# for i in range(0, len(x_train1), 400):
#     reduced_x_train = x_train1[i: i + 400]
#     cls_reduced_x_train_xlm, word_reduced_x_train_xlm = extract_xlm_r(list(reduced_x_train))
#     cls_reduced_x_train_torch = torch.cat(cls_reduced_x_train_xlm)
#     word_reduced_x_train_torch = torch.cat(word_reduced_x_train_xlm)
#     word_reduced_x_train_features = word_reduced_x_train_torch.cpu().detach().numpy()
#     cls_reduced_x_train_features = cls_reduced_x_train_torch.cpu().detach().numpy()
#     print(i)
#     if i != 0:
#         cls_x_train_features = np.concatenate((cls_x_train_features, cls_reduced_x_train_features), axis = 0)
#         word_x_train_features = np.concatenate((word_x_train_features, word_reduced_x_train_features), axis = 0)
#     else:
#         cls_x_train_features = cls_reduced_x_train_features
#         word_x_train_features = word_reduced_x_train_features
# #     reduced_y_train = np.asarray(reduced_y_train)
#     print(cls_x_train_features.shape)
#     print(word_x_train_features.shape)
#     del reduced_x_train, cls_reduced_x_train_xlm, cls_reduced_x_train_torch, cls_reduced_x_train_features, word_reduced_x_train_xlm, word_reduced_x_train_features, word_reduced_x_train_torch

In [13]:
# with open('train1_word_features_xlm.npy', 'wb') as f:
#     np.save(f, word_x_train_features)
    
# with open('train1_cls_features_xlm.npy', 'wb') as f:
#     np.save(f, cls_x_train_features)
    
# with open('train1_labels_xlm.npy', 'wb') as f:
#     np.save(f, y_train1)

In [14]:
# del word_x_train_features, cls_x_train_features

In [15]:
# for i in range(0, len(x_train2), 400):
#     reduced_x_train = x_train2[i: i + 400]
#     cls_reduced_x_train_xlm, word_reduced_x_train_xlm = extract_xlm_r(list(reduced_x_train))
#     cls_reduced_x_train_torch = torch.cat(cls_reduced_x_train_xlm)
#     word_reduced_x_train_torch = torch.cat(word_reduced_x_train_xlm)
#     word_reduced_x_train_features = word_reduced_x_train_torch.cpu().detach().numpy()
#     cls_reduced_x_train_features = cls_reduced_x_train_torch.cpu().detach().numpy()
#     print(i)
#     if i != 0:
#         cls_x_train_features = np.concatenate((cls_x_train_features, cls_reduced_x_train_features), axis = 0)
#         word_x_train_features = np.concatenate((word_x_train_features, word_reduced_x_train_features), axis = 0)
#     else:
#         cls_x_train_features = cls_reduced_x_train_features
#         word_x_train_features = word_reduced_x_train_features
# #     reduced_y_train = np.asarray(reduced_y_train)
#     print(cls_x_train_features.shape)
#     print(word_x_train_features.shape)
#     del reduced_x_train, cls_reduced_x_train_xlm, cls_reduced_x_train_torch, cls_reduced_x_train_features, word_reduced_x_train_xlm, word_reduced_x_train_features, word_reduced_x_train_torch

In [ ]:
# with open('train2_word_features_xlm.npy', 'wb') as f:
#     np.save(f, word_x_train_features)
    
# with open('train2_cls_features_xlm.npy', 'wb') as f:
#     np.save(f, cls_x_train_features)
    
# with open('train2_labels_xlm.npy', 'wb') as f:
#     np.save(f, y_train2)

In [ ]:
# del word_x_train_features, cls_x_train_features

In [ ]:
# for i in range(0, len(x_test), 400):
#     reduced_x_test = x_test[i: i + 400]
#     cls_reduced_x_test_xlm, word_reduced_x_test_xlm = extract_xlm_r(list(reduced_x_test))
#     cls_reduced_x_test_torch = torch.cat(cls_reduced_x_test_xlm)
#     word_reduced_x_test_torch = torch.cat(word_reduced_x_test_xlm)
#     word_reduced_x_test_features = word_reduced_x_test_torch.cpu().detach().numpy()
#     cls_reduced_x_test_features = cls_reduced_x_test_torch.cpu().detach().numpy()
#     print(i)
#     if i != 0:
#         cls_x_test_features = np.concatenate((cls_x_test_features, cls_reduced_x_test_features), axis = 0)
#         word_x_test_features = np.concatenate((word_x_test_features, word_reduced_x_test_features), axis = 0)
#     else:
#         cls_x_test_features = cls_reduced_x_test_features
#         word_x_test_features = word_reduced_x_test_features
# #     reduced_y_train = np.asarray(reduced_y_train)
#     print(cls_x_test_features.shape)
#     print(word_x_test_features.shape)
#     del reduced_x_test, cls_reduced_x_test_xlm, cls_reduced_x_test_torch, cls_reduced_x_test_features, word_reduced_x_test_xlm, word_reduced_x_test_features, word_reduced_x_test_torch

In [ ]:
# with open('test_cls_features_xlm.npy', 'wb') as f:
#     np.save(f, cls_x_test_features)
    
# with open('test_word_features_xlm.npy', 'wb') as f:
#     np.save(f, word_x_test_features)
    
# with open('test_labels_xlm.npy', 'wb') as f:
#     np.save(f, y_test)

### For only CLS features

In [11]:
import torch
from transformers import XLMRobertaTokenizer, XLMRobertaModel
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaModel.from_pretrained("xlm-roberta-base")
from tqdm import tqdm
def extract_xlm_r_cls_only(list_of_premises=[], list_of_hypothesis=[], batch_size=16):
    cls_hidden_states = []
    if len(list_of_premises)<batch_size: #go longer ones we should batch
        tknzed= tokenizer(list_of_premises, list_of_hypothesis, return_tensors="pt", padding=True)
        b=model(**tknzed)
        cls_hidden_states = torch.squeeze(b.last_hidden_state[:,0,:])
    else:
        for i in tqdm(range(0,len(list_of_premises),batch_size)):    
            tknzed= tokenizer(list_of_premises[i:min(len(list_of_premises),i+batch_size)], list_of_hypothesis[i:min(len(list_of_hypothesis),i+batch_size)], return_tensors="pt", padding=True)
            b=model(**tknzed)
            cls_hidden_states.append(torch.squeeze(b.last_hidden_state[:,0,:]))
#             print(b.last_hidden_state[:,0,:].shape)
#             print(b.last_hidden_state[:,1:,:].shape)
    return cls_hidden_states
    return cls_hidden_states

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
for i in range(0, len(x_train), 400):
    reduced_x_train = x_train[i: i + 400]
    cls_reduced_x_train = extract_xlm_r_cls_only(list(reduced_x_train['premise']), list(reduced_x_train['hypothesis']))
    cls_reduced_x_train_torch = torch.cat(cls_reduced_x_train)
    cls_reduced_x_train_features = cls_reduced_x_train_torch.cpu().detach().numpy()
    print(i)
    if i != 0:
        cls_x_train_features = np.concatenate((cls_x_train_features, cls_reduced_x_train_features), axis = 0)
    else:
        cls_x_train_features = cls_reduced_x_train_features
#     reduced_y_train = np.asarray(reduced_y_train)
    print(cls_x_train_features.shape)
    del reduced_x_train, cls_reduced_x_train, cls_reduced_x_train_torch, cls_reduced_x_train_features

100%|███████████████████████████████████████████| 25/25 [01:28<00:00,  3.56s/it]


0
(400, 768)


100%|███████████████████████████████████████████| 25/25 [00:15<00:00,  1.58it/s]


400
(800, 768)


100%|███████████████████████████████████████████| 25/25 [00:19<00:00,  1.27it/s]


...

80000
(80400, 768)


100%|███████████████████████████████████████████| 25/25 [00:12<00:00,  1.99it/s]


80400
(80800, 768)


100%|███████████████████████████████████████████| 19/19 [00:09<00:00,  2.03it/s]


80800
(81093, 768)


In [17]:
with open('train_cls_features_xlm_v3.npy', 'wb') as f:
    np.save(f, cls_x_train_features)
    
with open('train_labels_xlm_v3.npy', 'wb') as f:
    np.save(f, y_train)

In [18]:
for i in range(0, len(x_test), 400):
    reduced_x_test = x_test[i: i + 400]
    cls_reduced_x_test = extract_xlm_r_cls_only(list(reduced_x_test['premise']), list(reduced_x_test['hypothesis']))
    cls_reduced_x_test_torch = torch.cat(cls_reduced_x_test)
    cls_reduced_x_test_features = cls_reduced_x_test_torch.cpu().detach().numpy()
    print(i)
    if i != 0:
        cls_x_test_features = np.concatenate((cls_x_test_features, cls_reduced_x_test_features), axis = 0)
    else:
        cls_x_test_features = cls_reduced_x_test_features
#     reduced_y_train = np.asarray(reduced_y_train)
    print(cls_x_test_features.shape)
    del reduced_x_test, cls_reduced_x_test, cls_reduced_x_test_torch, cls_reduced_x_test_features

100%|███████████████████████████████████████████| 25/25 [00:14<00:00,  1.68it/s]


0
(400, 768)


100%|███████████████████████████████████████████| 25/25 [00:12<00:00,  1.98it/s]


400
(800, 768)


100%|███████████████████████████████████████████| 25/25 [00:13<00:00,  1.82it/s]


...

19200
(19600, 768)


100%|███████████████████████████████████████████| 25/25 [00:12<00:00,  1.96it/s]


19600
(20000, 768)


100%|███████████████████████████████████████████| 18/18 [00:08<00:00,  2.19it/s]


20000
(20274, 768)


In [19]:
with open('test_cls_features_xlm_v3.npy', 'wb') as f:
    np.save(f, cls_x_test_features)
    
with open('test_labels_xlm_v3.npy', 'wb') as f:
    np.save(f, y_test)

## Loading features

In [20]:
cls_x_train = np.load('train_cls_features_xlm_v3.npy', allow_pickle=True)
y_train = np.load('train_labels_xlm_v3.npy', allow_pickle=True)

In [21]:
cls_x_train.shape

(81093, 768)

In [22]:
y_train.shape

(81093,)

In [23]:
cls_x_test = np.load('test_cls_features_xlm_v3.npy', allow_pickle=True)
y_test = np.load('test_labels_xlm_v3.npy', allow_pickle=True)

In [24]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(cls_x_train)
simple_x_train = scaler.transform(cls_x_train)

simple_x_test =scaler.transform(cls_x_test)

In [25]:
simple_x_train.shape

(81093, 768)

In [26]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver = "liblinear")
clf.fit(simple_x_train, y_train)
pred = clf.predict(simple_x_test)
pred = classification_report(y_test, pred)
print(pred)

              precision    recall  f1-score   support

           0       0.51      0.52      0.52      6785
           1       0.47      0.47      0.47      6719
           2       0.49      0.48      0.49      6770

    accuracy                           0.49     20274
   macro avg       0.49      0.49      0.49     20274
weighted avg       0.49      0.49      0.49     20274



In [27]:
model = LinearSVC()

model.fit(simple_x_train, y_train)
pred = model.predict(simple_x_test)
pred = classification_report(y_test, pred)
print(pred)

              precision    recall  f1-score   support

           0       0.47      0.44      0.45      6785
           1       0.43      0.45      0.44      6719
           2       0.44      0.46      0.45      6770

    accuracy                           0.45     20274
   macro avg       0.45      0.45      0.45     20274
weighted avg       0.45      0.45      0.45     20274



/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [28]:
model = Perceptron()

model.fit(simple_x_train, y_train)
pred = model.predict(simple_x_test)
pred = classification_report(y_test, pred)
print(pred)

              precision    recall  f1-score   support

           0       0.43      0.42      0.43      6785
           1       0.41      0.38      0.39      6719
           2       0.40      0.43      0.42      6770

    accuracy                           0.41     20274
   macro avg       0.41      0.41      0.41     20274
weighted avg       0.41      0.41      0.41     20274



In [29]:
import xgboost as xgb
model = xgb.XGBClassifier()
model.fit(simple_x_train, y_train)
pred = model.predict(simple_x_test)
pred = classification_report(y_test, pred)
print(pred)

In [30]:
import torch
x_train_torch = torch.from_numpy(cls_x_train)
x_test_torch = torch.from_numpy(cls_x_test)

In [31]:
zipper = lambda x,y : list(zip(x,list(y)))

In [32]:
train_torch = zipper(x_train_torch, y_train)
test_torch = zipper(x_test_torch, y_test)

train_loader = torch.utils.data.DataLoader(train_torch, batch_size=16)
test_loader = torch.utils.data.DataLoader(test_torch, batch_size=16)

In [33]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module): 
    
    def __init__(self):
        super(Net, self).__init__() 
#         self.gru = nn.GRU(input_size=768, hidden_size=32, batch_first=True) 
        self.fc1 = nn.Linear(768,3)
#         self.fc2 = nn.Linear(256, 64)
#         self.fc3 = nn.Linear(128, 3)
        self.relu = nn.ReLU()

#         self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
#         out, hn = self.gru(x, torch.randn(1, len(x), 32))
#         out = self.fc1(self.relu(x))
#         out = self.fc2(self.relu(out))
        out = self.fc1(x)
        return out

In [34]:
linear_model = Net()
print(linear_model)

Net(
  (fc1): Linear(in_features=768, out_features=3, bias=True)
  (relu): ReLU()
)


In [35]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(linear_model.parameters(), lr=0.01)

In [36]:
# number of epochs to train the model
n_epochs = 5000

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    linear_model.train() # prep model for training
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = linear_model(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    linear_model.eval() # prep model for evaluation
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = linear_model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(test_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}' \
          .format(epoch+1, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}). \
        Saving model ...'.format(valid_loss_min, valid_loss))
        
        torch.save(linear_model.state_dict(), 'xlm_model_linear.pt')
        valid_loss_min = valid_loss
        

Epoch: 1 	Training Loss: 1.185740 	Validation Loss: 1.783295
Validation loss decreased (inf --> 1.783295).         Saving model ...
Epoch: 2 	Training Loss: 1.182679 	Validation Loss: 1.769638
Validation loss decreased (1.783295 --> 1.769638).         Saving model ...
Epoch: 3 	Training Loss: 1.179867 	Validation Loss: 1.757969
Validation loss decreased (1.769638 --> 1.757969).         Saving model ...
Epoch: 4 	Training Loss: 1.177343 	Validation Loss: 1.747956
Validation loss decreased (1.757969 --> 1.747956).         Saving model ...
Epoch: 5 	Training Loss: 1.175050 	Validation Loss: 1.739328
Validation loss decreased (1.747956 --> 1.739328).         Saving model ...
Epoch: 6 	Training Loss: 1.172947 	Validation Loss: 1.731865
Validation loss decreased (1.739328 --> 1.731865).         Saving model ...
Epoch: 7 	Training Loss: 1.171004 	Validation Loss: 1.725382
Validation loss decreased (1.731865 --> 1.725382).         Saving model ...
Epoch: 8 	Training Loss: 1.169197 	Validation 

Epoch: 61 	Training Loss: 1.131368 	Validation Loss: 1.665745
Validation loss decreased (1.666087 --> 1.665745).         Saving model ...
Epoch: 62 	Training Loss: 1.131042 	Validation Loss: 1.665403
Validation loss decreased (1.665745 --> 1.665403).         Saving model ...
Epoch: 63 	Training Loss: 1.130721 	Validation Loss: 1.665060
Validation loss decreased (1.665403 --> 1.665060).         Saving model ...
Epoch: 64 	Training Loss: 1.130405 	Validation Loss: 1.664716
Validation loss decreased (1.665060 --> 1.664716).         Saving model ...
Epoch: 65 	Training Loss: 1.130096 	Validation Loss: 1.664371
Validation loss decreased (1.664716 --> 1.664371).         Saving model ...
Epoch: 66 	Training Loss: 1.129791 	Validation Loss: 1.664026
Validation loss decreased (1.664371 --> 1.664026).         Saving model ...
Epoch: 67 	Training Loss: 1.129491 	Validation Loss: 1.663679
Validation loss decreased (1.664026 --> 1.663679).         Saving model ...
Epoch: 68 	Training Loss: 1.129197

Epoch: 121 	Training Loss: 1.118249 	Validation Loss: 1.643949
Validation loss decreased (1.644321 --> 1.643949).         Saving model ...
Epoch: 122 	Training Loss: 1.118100 	Validation Loss: 1.643576
Validation loss decreased (1.643949 --> 1.643576).         Saving model ...
Epoch: 123 	Training Loss: 1.117954 	Validation Loss: 1.643204
Validation loss decreased (1.643576 --> 1.643204).         Saving model ...
Epoch: 124 	Training Loss: 1.117808 	Validation Loss: 1.642832
Validation loss decreased (1.643204 --> 1.642832).         Saving model ...
Epoch: 125 	Training Loss: 1.117664 	Validation Loss: 1.642460
Validation loss decreased (1.642832 --> 1.642460).         Saving model ...
Epoch: 126 	Training Loss: 1.117522 	Validation Loss: 1.642088
Validation loss decreased (1.642460 --> 1.642088).         Saving model ...
Epoch: 127 	Training Loss: 1.117381 	Validation Loss: 1.641716
Validation loss decreased (1.642088 --> 1.641716).         Saving model ...
Epoch: 128 	Training Loss: 

Epoch: 180 	Training Loss: 1.111385 	Validation Loss: 1.622511
Validation loss decreased (1.622860 --> 1.622511).         Saving model ...
Epoch: 181 	Training Loss: 1.111294 	Validation Loss: 1.622162
Validation loss decreased (1.622511 --> 1.622162).         Saving model ...
Epoch: 182 	Training Loss: 1.111203 	Validation Loss: 1.621814
Validation loss decreased (1.622162 --> 1.621814).         Saving model ...
Epoch: 183 	Training Loss: 1.111113 	Validation Loss: 1.621467
Validation loss decreased (1.621814 --> 1.621467).         Saving model ...
Epoch: 184 	Training Loss: 1.111023 	Validation Loss: 1.621120
Validation loss decreased (1.621467 --> 1.621120).         Saving model ...
Epoch: 185 	Training Loss: 1.110934 	Validation Loss: 1.620774
Validation loss decreased (1.621120 --> 1.620774).         Saving model ...
Epoch: 186 	Training Loss: 1.110845 	Validation Loss: 1.620428
Validation loss decreased (1.620774 --> 1.620428).         Saving model ...
Epoch: 187 	Training Loss: 

Epoch: 239 	Training Loss: 1.106850 	Validation Loss: 1.603097
Validation loss decreased (1.603405 --> 1.603097).         Saving model ...
Epoch: 240 	Training Loss: 1.106785 	Validation Loss: 1.602790
Validation loss decreased (1.603097 --> 1.602790).         Saving model ...
Epoch: 241 	Training Loss: 1.106721 	Validation Loss: 1.602483
Validation loss decreased (1.602790 --> 1.602483).         Saving model ...
Epoch: 242 	Training Loss: 1.106657 	Validation Loss: 1.602178
Validation loss decreased (1.602483 --> 1.602178).         Saving model ...
Epoch: 243 	Training Loss: 1.106593 	Validation Loss: 1.601873
Validation loss decreased (1.602178 --> 1.601873).         Saving model ...
Epoch: 244 	Training Loss: 1.106530 	Validation Loss: 1.601569
Validation loss decreased (1.601873 --> 1.601569).         Saving model ...
Epoch: 245 	Training Loss: 1.106467 	Validation Loss: 1.601265
Validation loss decreased (1.601569 --> 1.601265).         Saving model ...
Epoch: 246 	Training Loss: 

...

Epoch: 4763 	Training Loss: 1.075751 	Validation Loss: 1.477912
Validation loss decreased (1.477913 --> 1.477912).         Saving model ...
Epoch: 4764 	Training Loss: 1.075750 	Validation Loss: 1.477911
Validation loss decreased (1.477912 --> 1.477911).         Saving model ...
Epoch: 4765 	Training Loss: 1.075749 	Validation Loss: 1.477910
Validation loss decreased (1.477911 --> 1.477910).         Saving model ...
Epoch: 4766 	Training Loss: 1.075747 	Validation Loss: 1.477909
Validation loss decreased (1.477910 --> 1.477909).         Saving model ...
Epoch: 4767 	Training Loss: 1.075746 	Validation Loss: 1.477908
Validation loss decreased (1.477909 --> 1.477908).         Saving model ...
Epoch: 4768 	Training Loss: 1.075745 	Validation Loss: 1.477908
Validation loss decreased (1.477908 --> 1.477908).         Saving model ...
Epoch: 4769 	Training Loss: 1.075744 	Validation Loss: 1.477907
Validation loss decreased (1.477908 --> 1.477907).         Saving model ...
Epoch: 4770 	Trainin

Epoch: 4822 	Training Loss: 1.075683 	Validation Loss: 1.477835
Validation loss decreased (1.477838 --> 1.477835).         Saving model ...
Epoch: 4823 	Training Loss: 1.075682 	Validation Loss: 1.477834
Validation loss decreased (1.477835 --> 1.477834).         Saving model ...
Epoch: 4824 	Training Loss: 1.075681 	Validation Loss: 1.477832
Validation loss decreased (1.477834 --> 1.477832).         Saving model ...
Epoch: 4825 	Training Loss: 1.075680 	Validation Loss: 1.477830
Validation loss decreased (1.477832 --> 1.477830).         Saving model ...
Epoch: 4826 	Training Loss: 1.075678 	Validation Loss: 1.477829
Validation loss decreased (1.477830 --> 1.477829).         Saving model ...
Epoch: 4827 	Training Loss: 1.075677 	Validation Loss: 1.477826
Validation loss decreased (1.477829 --> 1.477826).         Saving model ...
Epoch: 4828 	Training Loss: 1.075676 	Validation Loss: 1.477825
Validation loss decreased (1.477826 --> 1.477825).         Saving model ...
Epoch: 4829 	Trainin

Epoch: 4881 	Training Loss: 1.075616 	Validation Loss: 1.477748
Validation loss decreased (1.477749 --> 1.477748).         Saving model ...
Epoch: 4882 	Training Loss: 1.075615 	Validation Loss: 1.477746
Validation loss decreased (1.477748 --> 1.477746).         Saving model ...
Epoch: 4883 	Training Loss: 1.075614 	Validation Loss: 1.477746
Validation loss decreased (1.477746 --> 1.477746).         Saving model ...
Epoch: 4884 	Training Loss: 1.075613 	Validation Loss: 1.477744
Validation loss decreased (1.477746 --> 1.477744).         Saving model ...
Epoch: 4885 	Training Loss: 1.075612 	Validation Loss: 1.477743
Validation loss decreased (1.477744 --> 1.477743).         Saving model ...
Epoch: 4886 	Training Loss: 1.075611 	Validation Loss: 1.477741
Validation loss decreased (1.477743 --> 1.477741).         Saving model ...
Epoch: 4887 	Training Loss: 1.075610 	Validation Loss: 1.477740
Validation loss decreased (1.477741 --> 1.477740).         Saving model ...
Epoch: 4888 	Trainin

Epoch: 4940 	Training Loss: 1.075551 	Validation Loss: 1.477665
Validation loss decreased (1.477666 --> 1.477665).         Saving model ...
Epoch: 4941 	Training Loss: 1.075550 	Validation Loss: 1.477663
Validation loss decreased (1.477665 --> 1.477663).         Saving model ...
Epoch: 4942 	Training Loss: 1.075549 	Validation Loss: 1.477661
Validation loss decreased (1.477663 --> 1.477661).         Saving model ...
Epoch: 4943 	Training Loss: 1.075548 	Validation Loss: 1.477659
Validation loss decreased (1.477661 --> 1.477659).         Saving model ...
Epoch: 4944 	Training Loss: 1.075547 	Validation Loss: 1.477658
Validation loss decreased (1.477659 --> 1.477658).         Saving model ...
Epoch: 4945 	Training Loss: 1.075546 	Validation Loss: 1.477657
Validation loss decreased (1.477658 --> 1.477657).         Saving model ...
Epoch: 4946 	Training Loss: 1.075544 	Validation Loss: 1.477655
Validation loss decreased (1.477657 --> 1.477655).         Saving model ...
Epoch: 4947 	Trainin

Epoch: 4999 	Training Loss: 1.075487 	Validation Loss: 1.477586
Validation loss decreased (1.477588 --> 1.477586).         Saving model ...
Epoch: 5000 	Training Loss: 1.075486 	Validation Loss: 1.477586
Validation loss decreased (1.477586 --> 1.477586).         Saving model ...


In [37]:
linear_model = Net()
linear_model.load_state_dict(torch.load("xlm_model_linear.pt"))
linear_model.eval()

pred_list = torch.zeros(0, dtype=torch.long)
target_list = torch.zeros(0, dtype=torch.long)
train_count = 0
count = 0

train_pred_list = torch.zeros(0, dtype=torch.long)
train_target_list = torch.zeros(0, dtype=torch.long)

for data, target in train_loader:
    output = linear_model(data)
    _, preds = torch.max(output, 1) 
    train_pred_list = torch.cat([train_pred_list, preds.view(-1)])
    train_target_list = torch.cat([train_target_list, target.view(-1)])
#     for x in range(16):
#         if preds[x] == target[x]: 
#             train_count+=1
            
# print("Training Accuracy =", train_count / len(train_loader.dataset))
train_result = classification_report(train_pred_list.numpy(), 
                                     train_target_list.numpy())
print("Training Classification report: \n", train_result)
            
for data, target in test_loader:
    output = linear_model(data)
    _, preds = torch.max(output, 1) 
    pred_list = torch.cat([pred_list, preds.view(-1)])
    target_list = torch.cat([target_list, target.view(-1)])
#     for x in range(16):
#         if preds[x] == target[x]: 
#             count+=1

# print("Testing Accuracy =", count / len(test_loader.dataset))

result = classification_report(pred_list.numpy(), target_list.numpy())
print("Testing Classification report: \n", result)

Training Classification report: 
               precision    recall  f1-score   support

           0       0.99      0.34      0.50     79752
           1       0.02      0.57      0.03       736
           2       0.01      0.58      0.03       605

    accuracy                           0.34     81093
   macro avg       0.34      0.50      0.19     81093
weighted avg       0.98      0.34      0.50     81093

Testing Classification report: 
               precision    recall  f1-score   support

           0       0.99      0.34      0.50     19943
           1       0.01      0.52      0.03       169
           2       0.01      0.51      0.02       162

    accuracy                           0.34     20274
   macro avg       0.34      0.45      0.18     20274
weighted avg       0.97      0.34      0.49     20274

